In [4]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pprint import pprint
from tools.settings import settings

## Example from mtscomp for compression and decompression
```python
import numpy as np
from mtscomp.mtscomp import compress, decompress

# Compress a .bin file into a pair .cbin (compressed binary file) and .ch (JSON file).
compress('data.bin', 'data.cbin', 'data.ch', sample_rate=20000., n_channels=256, dtype=np.int16)
# Decompress a pair (.cbin, .ch) and return an object that can be sliced like a NumPy array.
arr = decompress('data.cbin', 'data.ch')
X = arr[start:end, :]  # decompress the data on the fly directly from the file on disk
arr.close()  # Close the file when done
```

## setup

#### set paths

In [9]:
paths = settings.paths
experiment = settings.experiment

raw_drive = paths.drive
expt_folder = experiment.dir
batch_folder = raw_drive / expt_folder / 'temp_storage'
print(f'Looking for recordings in:\n\t{batch_folder}')

target_folder = batch_folder.parent / 'compressed_recordings'
target_folder.mkdir(exist_ok=True)
print(f'\nCompressed recordings will be saved to:\n\t{target_folder}')


Looking for recordings in:
	/mnt/fasthd/4_Behavioral_Sensitization/temp_storage

Compressed recordings will be saved to:
	/mnt/fasthd/4_Behavioral_Sensitization/compressed_recordings


set subject / recording pairs to process

In [10]:
recording_pairs = experiment.recordings
print("Processing the following recordings:")
pprint(recording_pairs, indent=4, width=10)

Processing the following recordings:
{   'BS03_R1': {   'concatenate': False},
    'BS03_R2': {   'concatenate': False},
    'BS04_R1': {   'concatenate': False},
    'BS04_R2': {   'concatenate': False},
    'BS04_R3': {   'concatenate': False},
    'BS05_R1': {   'concatenate': False}}


> If just wanting to create new project folder structures, skip to  *"Once compressions are done..."* section.
> 

## compress recordings

**single recording**  
To compress a single recording, assign `recording_name` to one of the entries in `run_config.toml` under `[experiment.recordings]` header, for example "NP02_R1".  
Run the `compress_recordings` cell afterwards to compress the recording to `target_folder`.  

**batch compress**  
To batch compress, simply leave `recording_name` as `None`, then run the `compress_recordings` cell to compresse all recordings in `recording_pairs` to `target_folder`.

set parallel processing parameters

In [ ]:
num_cores = os.cpu_count()
job_kwargs=dict(
    n_threads=round(num_cores*0.8),
    chunk_duration=5,
)
print("\nParallel Job parameters:")
pprint(job_kwargs, indent=4)


Parallel Job parameters:
{'chunk_duration': 5, 'n_threads': 10}


In [ ]:
# change to other than None to compress specific raw recording (including multiple segments)
# e.g. "NP02_R1", or keep as None to batch compress all recordings in recording_pairs
recording_name = None

In [ ]:
from tools.compression import compress_recordings

if recording_name is not None:
    # compress single recording session
    properties = recording_pairs[recording_name]
    print(f'---processing single recording')
    compress_recordings(
        {recording_name: properties},
        batch_folder,
        target_folder,
        job_kwargs=job_kwargs
    )
else:
    # compress all recording sessions
    print(f'---processing all recordings in recording_pairs...')
    compress_recordings(
        recording_pairs,
        batch_folder,
        target_folder,
        job_kwargs=job_kwargs
    )


## once compressions are done...
If you haven't yet, set up experiment folder structure for each recording using the `set_up_folders` tool.  

Currently there are two options:   
* *Command-line tool*
    * Useful to set up structure for one session, or to have some more control over output.
    * If you installed with uv, should be able to use the following to see basic usage, see README for further instructions.  
    
        ```bash
        uv run set_up_folders --help
        ```
* *Set up default structures (histology & recording templates)*
    * For all recordings in `recording_pairs`, or single recording if `recording_name` is other than `None`.
    * See further instructions below

In [14]:
from pathlib import Path

# Define the base path where the project folders for each subject will be located
    # this should or will contain subject folders like 'NP02_R1', etc.
# For our projects, for each experiment `data_dir` by default is "1_Recordings"
project_base_path = raw_drive / expt_folder / paths.data_dir

# set to True to see what folders would be created without actually creating them
in_debug_mode = False


In [15]:
from tools.set_up_folders import setup_folders_from_dict

recording_name = None

if recording_name is not None:
    print(f'---setting up folders for recording {recording_name}...')
else:
    print(f'---setting up folders for all recordings in recording_pairs...')
setup_folders_from_dict(
    recording_pairs,
    base_path=project_base_path,
    debug=in_debug_mode
)



---setting up folders for all recordings in recording_pairs...
Setting up folder structures under: /mnt/fasthd/4_Behavioral_Sensitization/1_Recordings
--- Processing session: BS03_R1 ---
Creating full folder structure for: /mnt/fasthd/4_Behavioral_Sensitization/1_Recordings/BS03/0_histology
Folder structure created successfully under '/mnt/fasthd/4_Behavioral_Sensitization/1_Recordings/BS03'
0_histology/
├── 0_unregistered_downsampled/
├── 1_registered_napari/
├── 2_transformed_stacks/
└── 3_lasagna_tracks/

You can find the new folders in: /mnt/fasthd/4_Behavioral_Sensitization/1_Recordings/BS03/0_histology

Creating folder structure for recording session: BS03_R1
Folder structure created successfully for BS03_R1:
BS03_R1/
├── 0_raw_compressed/
├── 1_histology_alignment/
├── 2_processed/
│   ├── kilosort4/
│   ├── metrics/
│   └── aligned_outputs/
└── 3_datasets/
    ├── sync_data/
    ├── metadata/
    └── plots/

You can find the new session folder in: /mnt/fasthd/4_Behavioral_Sensi

[link](#a-heading)

## a heading